In [3]:
import os
import shutil
import time
import pandas as pd
from tqdm import tqdm      #tqdm(filelist, total = len(file_list), position=0, leave=True)
import re
from pathlib import Path
import traceback
from os.path import join
import file_function as ff
from datetime import datetime
from upload_v2 import dict_refer, write_log_csv, re_name, file_listFnc
import pickle

path_log = r'D:\0.전산\1.진행중과업\파일서버개편\log'

In [ ]:
path = r'C:\Users\SL\Desktop\test'
ff.moveFilesToRootFolderAndRmDirs(path)

### 파일명 작성 지침 변경에 따른 부산 수정

##### 파일 전체 옮기기

In [ ]:
path = r'D:\부산 정리_v1'
dst_root = r'D:\부산 정리_v2'
ff.copy_all(path, dst_root)

##### '보증인', 집행권원, 강제집행 등 문서종류 및 보증인 수정

In [3]:
path = r'\\192.168.0.75\스캔파일\새 스캔파일(업로드)'
# filelist = os.walk(path)
file_list = [f.name for f in os.scandir(path) if f.is_file()]

for f in tqdm(file_list) :
    if f != "Thumbs.db" :
        new_name = f

        if re.search("판결문|지급명령|이행권고|화해권고", new_name) :
            new_name = re.sub("판결문|지급명령|이행권고|화해권고", "집행권원", new_name)

        elif re.search("타채|결정문", new_name) :
            new_name = re.sub("타채|결정문", "강제집행", new_name)

        elif re.search("채권양도통지서", new_name) :
            new_name = re.sub("채권양도통지서", "양도통지서", new_name)
        
        elif re.search("[_]개회", new_name) :
            new_name = re.sub("_개회", "_개인회생", new_name)

        elif re.search("[_]신복", new_name) :
            new_name = re.sub("_신복", "_신용회복", new_name)

        if re.search("[_]+보증인", new_name) :
            split_list = re.sub("[_]+보증인", "", new_name).split("_")
            split_list[1] = "보증인" + " "+ split_list[1]
            new_name = "_".join(split_list)

        


        if new_name != f : 
                                        ########
            ff.re_name(join(path,f), join(path, new_name))
                                        ########
    

  0%|          | 0/24184 [00:00<?, ?it/s]

##### 사건번호 다시 넣기

In [ ]:
v2_dict = ff.fileInfoDict(r"D:\부산 정리_v2")

In [ ]:
len(v2_dict) #{채무자키 : {size : fullpath of files in 부산정리_v2}}

In [ ]:
# 사건번호 있는 파일 목록 작성
p_key = re.compile(r"([\d]{8})[\D]") #match
p_event = re.compile(r"[\D]([\d]{4}[ㄱ-ㅎ가-힣]{1,3}[0-9]+)")
p_extension = re.compile('(jpeg|jpg|bmp|gif|pdf|png|tif|xps)$', re.I)

no_key_error = []
no_file_error = []
result = [] #4740개 나와야


path = r'D:\1.다운로드\__부산 공유폴더_원본'
filelist = os.walk(path)

In [ ]:
index = 0
for root, __dirs__, files in tqdm(filelist) :
    
    for f in files :
        index += 1
        if (p_extension.search(f) != None) : #스캔본인 경우
            
            # [주의] 사건번호에 띄어쓰기 없어야 하므로 파일명에서 띄어쓰기 제거
            nospace_f = re.sub(" ", "", f)

            if p_event.search(nospace_f) != None : # 사건번호 있으면
                if p_key.search(f) != None : # 원본이라 채무자키가 젤 앞에 없는 경우도 많다.
                    
                    key = str(p_key.search(f).group(1)) # str() 유의
                    size = str(os.path.getsize(join(root, f))) # str() 유의
                    event = p_event.search(nospace_f).group(1) #nospace_f, group(1) 유의
                    
                    try :
                        src = v2_dict[key][size] #변경할 파일 풀경로
                        
                        src_dir = os.path.split(src)[0]
                        src_filename = os.path.split(src)[1]
                        src_stem = os.path.splitext(src_filename)[0]
                        src_ext = os.path.splitext(src_filename)[1]
                        
                        # 사건번호 넣기
                        src_split_list = src_stem.split("_")
                        src_split_list.insert(3, event) 
                        
                        dst_filename = "_".join(src_split_list) + src_ext
                        dst = join(src_dir, dst_filename)
                        
                        result.append(ff.re_name(src, dst))

                    except :
                        no_file_error.append([key, size, event, join(root, f)])
                        continue #아래 코드 실행 안 됨
                else :
                    no_key_error.append(join(root, f))
            else : pass
        else : pass

print(index)

In [ ]:
print(result)

##### 양통 차수, 집행권원 재도

In [ ]:
path = r'D:\부산 정리_v2'
filelist = os.walk(path)
result = []

p_재도 = re.compile("_재도$")
p_차수 = re.compile("_([0-9]차)$")

In [ ]:
for root, __dirs__, files in filelist :

    for f in files :

        src = join(root, f)
        stem = os.path.splitext(f)[0]
        ext = os.path.splitext(f)[1]

        if p_재도.search(stem) :
            
            stem = p_재도.sub("", stem)
            split_list = stem.split("_")
            split_list[2]= split_list[2]+" "+"재도"

            dst_filename = "_".join(split_list) + ext
            dst = join(root, dst_filename)
            result.append(ff.re_name(src, dst))
        
        elif p_차수.search(stem) :

            차수 = p_차수.search(stem).group(1)
            stem = p_차수.sub("", stem)
            split_list = stem.split("_")
            split_list[2] = split_list[2]+ " " + 차수

            dst_filename = "_".join(split_list) + ext
            dst = join(root, dst_filename)
            result.append(ff.re_name(src, dst))

print(len(result), "개의 이름 변경")

##### 재산조회, 녹취록 (수작업 함)

### 체크하기

##### >>> 채무자키만

In [ ]:
path = r'D:\부산 정리'
os.chdir(path)
file_list = os.walk(path)

p_key = re.compile(r'[\d]{8}[\D]')
p_notkey = re.compile('[0-9]{9,}')
error = []

for r, d, files in tqdm(file_list) :
    for f in files :
        if f != "Thumbs.db":
            if p_key.match(f) == None :
                error.append(f)
print(error)

##### >>>문서종류만

In [ ]:
p_docu = re.compile('원인서류|채권양도통지서|판결문|지급명령|이행권고|화해권고|타채|결정문|등본|초본|개회|신복|파산|외국인증명')

path = r'D:\부산 정리'
file_list = os.walk(path)
error_short = []
error_docu = []

for r, d, files in tqdm(file_list) :
    for f in files :
        if f != "Thumbs.db":
            splitname = f.split("_")
            if len(splitname)<3 :
                error_short.append(f)
            elif p_docu.search(splitname[2]) == None :
                error_docu.append(f)
print("short :", *error_short, sep="\n")
print("docu :", *error_docu, sep="\n")

### 돌발상황 처리

##### 이름 중간에 들어간 _


In [ ]:
### 등,초본이 초본을 포함해버린다. 이거 수정해야
path = r'D:\부산 정리'

p_docu = re.compile(
    '원인서류|채권양도통지서|판결문|지급명령|이행권고|화해권고|타채|결정문|등본|초본|개회|신복|파산|외국인증명')
p_ = re.compile("_")

os.chdir(path)
file_list = os.walk(path)
fixed = []
count = 0

for r, d, files in tqdm(file_list):
    for f in files:
        if f != "Thumbs.db":
            new_name = ""
            if p_docu.search(f):
                st = p_docu.search(f).start() - 1
                new_name = f[:9] + p_.sub(" ", f[9:st]) + f[st:]  # 앞뒤 언더바 말고 중간에 언더바

                if f != new_name:
                    os.rename(r+"/"+f, r+"/"+new_name)
                    count += 1
                    fixed.append([f, new_name])
                    

print(count, "개의 파일 이름 변경 완료")
print(*fixed, sep="\n")

##### 잘린 외국인 이름 정정

In [ ]:
# 파일명에 보증인 키워드가 있는지만 보고 그거에 맞춰서 ... 

# 채무자조회파일 dict로 불러오기
dict = ff.debtorInfoDict()

path = r'D:\부산 정리'
file_list = os.walk(path)

p_eng = re.compile("[a-zA-Z]+")
p_etc = re.compile("보증인")

count = 0

for root, dir, files in tqdm(file_list) : 
    for f in files : 

        if f != "Thumbs.db":
            new_name = ""
            
            spl_name = f.split("_", maxsplit=2)

            key = spl_name[0]
            debtor_name = spl_name[1]
            others = spl_name[2]

            if p_eng.search(debtor_name) : # 이름에 영어 있는 경우
                if p_etc.search(others) : # 보증인 키워드 있는 경우
                    new_name = key +"_" + str(dict[key].보증인성명)+"_"+ others
                else : # 보증인 키워드 없는 경우
                    new_name = key +"_" + str(dict[key].성명)+"_"+others
                    
                if new_name != f :
                    src = join(root, f)
                    dst = join(root, new_name)
                    ff.re_name(src, dst)
                    count += 1
print(count, "개의 파일명 변경 완료")
                  

##### 채무자키로 보증인 이름 넣기

In [ ]:
path = r'D:\key\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\18.(9017) 메리츠캐피탈_솔림헬프\3.채권양도통지서\2차\스캔_보증인'
for f in tqdm(os.listdir(path)) :
    key = f[:8]
    grt = dict_refer[key][3]
    new_name = key + " " + grt + " " + f[8:]
    try :
        os.rename(join(path, f), join(path, new_name))
    except :
        print(f)
        pass

100%|██████████| 174/174 [00:05<00:00, 29.92it/s]


### 수정파일 재작업

##### (날짜4~8자리) -> 6자리

In [ ]:
PATH = r'\\192.168.0.75\솔림헬프'
PATH_LOG_SUCCESS = r'\\192.168.0.75\스캔파일\스캔파일 log\success'
PATH_LOG_FAIL = r'\\192.168.0.75\스캔파일\스캔파일 log\fail'


# PATH = r'C:\Users\SL\Desktop\test'
# PATH_LOG_SUCCESS = r'C:\Users\SL\Desktop\test\log\success'
# PATH_LOG_FAIL = r'C:\Users\SL\Desktop\test\log\fail'



file_list = os.walk(PATH)
log = []
error = []
p_day211 = re.compile(r'[\s_]?\((\d\d)(\d)(\d)\)')
p_day212 = re.compile(r'[\s_]?\((\d\d)(\d)(\d\d)\)')
p_day411 = re.compile(r'[\s_]?\(20(2\d)(\d)(\d)\)')
p_day412 = re.compile(r'[\s_]?\(20(2\d)(\d)(\d\d)\)')

for root, __dirs__, files in tqdm(file_list) :
    try :
        for f in files :
            n = f
            if p_day211.search(f) :
                n = p_day211.sub('_\g<1>0\g<2>0\g<3>', n)
                #print("211", f, n)
            elif p_day212.search(f) :
                n = p_day212.sub('_\g<1>0\g<2>\g<3>', n)
                #print("212", f, n)
            elif p_day411.search(f) :
                n = p_day411.sub('_\g<1>0\g<2>0\g<3>', n)
                #print("411", f, n)
            elif p_day412.search(f) :
                n = p_day412.sub('_\g<1>0\g<2>\g<3>', n)
                #print("412", f, n)
            else :pass

            if n != f :
                log.append(ff.re_name(join(root,f), join(root,n)))


    except Exception as e:
        print("===================================")
        print(datetime.today().strftime("%H:%M:%S"))
        print(f, e.__class__, e, sep=" : ")
        print(traceback.format_exc())
        error.append([f, e.__class__, e])
        continue # 반복문 계속 돌아

ff.write_log_csv(log, PATH_LOG_SUCCESS)
ff.write_log_csv(error, PATH_LOG_FAIL)



##### 양통 차수 잘린거 복구

In [4]:
df = pd.read_csv(r"\\192.168.0.75\스캔파일\스캔파일log\success\20220921 130138_9059.csv", header=None)
df.head()

,0,1,2,3
0,/volume1/스캔파일/새 스캔파일(업로드)/나우리,20407905 조경미양도통지서.pdf,20407905_조경미_양도통지서.pdf,/volume1/솔림헬프/2.양도통지서/한울가람/20407905
1,/volume1/스캔파일/새 스캔파일(업로드)/나우리,20407941 강성준 양도통지서 20191008.pdf,20407941_강성준_양도통지서_191008.pdf,/volume1/솔림헬프/2.양도통지서/한울가람/20407941
2,/volume1/스캔파일/새 스캔파일(업로드)/나우리,20407937 전태수-헬프양도통지서.pdf,20407937_전태수 헬프_양도통지서.pdf,/volume1/솔림헬프/2.양도통지서/한울가람/20407937
3,/volume1/스캔파일/새 스캔파일(업로드)/나우리,20407941 강성준양도통지서이력 20191008.pdf,20407941_강성준_양도통지서_이력_191008.pdf,/volume1/솔림헬프/2.양도통지서/한울가람/20407941
4,/volume1/스캔파일/새 스캔파일(업로드)/나우리,20407959 박은혜(78) 순차적양도통지서.pdf,20407959_박은혜(78) 순차적_양도통지서.pdf,/volume1/솔림헬프/2.양도통지서/한울가람/20407959


In [11]:
import os
import pandas as pd
from os.path import join

df = pd.read_csv("/volume1/스캔파일/스캔파일log/success/20220921 130138_9059.csv", header=None)

dst_d = '/volume1/스캔파일/새 스캔파일(업로드)/나우리'

for i in tqdm(range(len(df.index))) :
    try :
        src_d = df.iloc[i,3]
        src_f = df.iloc[i,2]
        dst_f = df.iloc[i,1]
        os.rename(join(src_d, src_f), join(dst_d, dst_f))
    except :
        pass
    
print(i, "개 파일이동")

    
    

  0%|          | 6/9059 [00:00<00:01, 6017.65it/s]

0
1
2
3
4
5


### 구스캔폴더 작업

##### 상위 폴더로 파일 끄집어내기

In [3]:
path = r'D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류'
addInfo = ""

In [13]:
from file_function import re_name


temp = []
for root, dir, files in tqdm(os.walk(path)) :
    for f in files :
        try :
            # 그냥 꺼내기 
            # os.rename(join(root, f), join(path, root, f))
            
            # 상위 폴더명을 파일이름에 붙여 꺼내기
            dir = root.split(os.path.sep)[-1]
            new_name = dir + " " + f
            temp.append(re_name(join(root, f), join(path, new_name)))
        except :
            print("======================")
            print(f)
            pass
print(f"{len(temp)}건 파일 이동 완료")
write_log_csv(temp, path, addInfo)

400it [01:17,  5.19it/s]

7377건 파일 이동 완료


##### 빈 폴더 지우기

In [4]:
#path = r'D:\key\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\25.퀀텀_솔림헬프\5.매각사측 수령 스캔본\매각사측 원인서류 스캔본'
cnt = 0
for root, dirs, files in tqdm(os.walk(path)) :
	# 내for문 없이 root바로 remove해도 되는데, 이렇게 하면 수가 조금 줄어드네.
        for dir in dirs :
            try :
                #os.removedirs(os.path.join(root, dir))
                os.rmdir(os.path.join(root, dir))
                cnt += 1
            except : # 파일 있는 경우
                print(dir)
                pass
print(f"{cnt}개의 빈 폴더 삭제")

1it [00:00,  6.98it/s]

399개의 빈 폴더 삭제


##### 관리자키 > 채무자키

In [4]:
# 채무자조회에서 원하는 것만 불러와서 dict로 만들기
path_df = r'D:\0.전산\1.진행중과업\파일서버개편\파일\extraKey\dict모음' 
with open(join(path_df, "account.pkl"), "rb") as pkl : dict_account = pickle.load(pkl)
with open(join(path_df, "admin.pkl"), "rb") as pkl : dict_admin = pickle.load(pkl)
with open(join(path_df, "note.pkl"), "rb") as pkl : dict_note = pickle.load(pkl)
with open(join(path_df, "rgt.pkl"), "rb") as pkl : dict_rgt = pickle.load(pkl)

In [15]:
# 파일 읽고 DICT 만들기
df = pd.read_excel(r'D:\0.전산\1.진행중과업\파일서버개편\파일\extraKey\펜타일련번호.xlsx')
df
# print(len(df.index))
# df.fillna(0, inplace=True)
# print(len(df.index))
# df.head()

,일련번호,주민등록번호
0,8101001522,520406-2775113
1,6001000152,650112-1541011
2,6001000176,320220-1002311
3,6001000432,700807-2566811
4,6001000454,730217-1691715
...,...,...
1041,9001001124,650325-1465213
1042,9001001148,670224-1658814
1043,9001001167,680620-1628611
1044,9001001315,761117-1357511


In [27]:
penta = {}
for i, v in df.iterrows() :
    penta[str(v.일련번호)] = v.주민등록번호.replace("-", "")

In [28]:
# with open(r"D:\0.전산\1.진행중과업\파일서버개편\파일\extraKey\dict모음\펜타.pkl", 'wb') as pkl :
#     pickle.dump(penta, pkl)

In [46]:
# dict_TYM = {}
# for k, v in df_TYM.iterrows() :
#     dict_TYM[str(v["양도통지서 파일명 수정_"]).strip()] = {"관리자키" : str(int(v.관리자키)), "구분" : v.구분, "보증인" : v["양도통지서 파일명 수정"]}
# len(dict_TYM)

3515

In [84]:
# if dict_TYM[n]["구분"] == "티와이(차주)" :
#     pass
# else :
#     name = re.search("[가-힣]+", dict_TYM[n]["보증인"]).group()
# print(name)
# # 파일 읽고 DICT 만들기 [끝]

주현수


In [101]:
# with open(join(path_df, "TYM.pkl"), "rb") as pkl : dict_TYM = pickle.load(pkl) #하나의 파일명이 여러개의 계좌에 있고 마지막 계좌로 입력되어있다. 하나의 파일이 여러 채무자키에 연결되었다고 보기는 어려우니 채무자키는 상관없을 것이다.
# -1지우고 stem 오른쪽 공백도 지웠다.

In [94]:
##### 파일명 > 계좌번호 등 > 채무자키
##############################################
path = r'D:\extraKey\광주 공유폴더\광주대출원장폴더-작업중'
noWhere = r'D:\전산부재\광주 광주대출원장-작업중'
dict_refer = dict_admin
add = " 원인서류"
##############################################

no_key = []

p_extension = re.compile('(jpeg|jpg|bmp|gif|pdf|png|tif)$', re.I)
file_list = [f.name for f in os.scandir(path) if f.is_file() and (re.search(p_extension, f.name))]
total=len(file_list)
cnt = 0
#p = re.compile("\s-1|a$") # 뒤에 꼬리표 떼기
p = re.compile("\D*")
p0 = re.compile("^0*")
for f in tqdm(file_list, total=total) :
    stem, ext = os.path.splitext(f)
    n = stem.strip()
    n = p0.sub("", n)
    n = p.sub("", n)
    n = re.sub("000$", "100", n)
    #nStem = stem.rstrip()
    #nStem = p.sub("", nStem)
    #nExt = ext.upper() # 대문자 변경
    #n = nStem + nExt
    name, param = "", ""

    try :
        # dic = dict_TYM[n]
        # param = dict_TYM[n]["관리자키"]
        
        if re.search("[가-힣]+", dic["보증인"]) :
            name = re.search("[가-힣]+", dic["보증인"]).group()
        else :
            name = "보증인"
    except :
        no_key.append([f, "no_admin"])
        continue
    try :
        key = dict_refer[param] 
    except :
        os.rename(join(path, f), join(noWhere, f))
        no_key.append([f, "no_key"])
        continue

    try :
        new_name = key + " " +name + " " + stem + add + ext # -1이면 중복이 있다는 소리니 새파일명은 원래 stem을 사용하자
        os.rename(join(path, f), join(path, new_name))
        cnt += 1
    except Exception as e :
        print(f, e.__class__, e, sep=" : ")
        pass
    
loss = total - cnt
print(f'{total}개 중 {cnt}개 파일 이름 변경 완료. {loss}개 확인 요망')
write_log_csv(no_key, path)

100%|██████████| 799/799 [00:00<00:00, 72880.17it/s]

799개 중 32개 파일 이름 변경 완료. 767개 확인 요망


In [31]:
##### 일련번호 > 사업자/주민번호 > 채무자키
##############################################
path = r'D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류'
addInfo = "광주 모음 대출원장 케이에스 펜타"
noWhere = join(r'D:\전산부재', addInfo)
dict_refer = dict_rgt
add = ""
dict_sn = penta
##############################################
# p_rgt1 = re.compile(r'((?<=\D)|\b)\d{13}((?=\D)|\b)') # 13자리
# p_rgt2 = re.compile(r'((?<=\D)|\b)(\d{3})-?(\d{2})-?(\d{5})((?=\D)|\b)') # 10자리
no_key = []
success = []
fail = []
noWhere_cnt = 0

p_extension = re.compile('(jpeg|jpg|bmp|gif|pdf|png|tif)$', re.I)
file_list = [f.name for f in os.scandir(path) if f.is_file() and (re.search(p_extension, f.name))]
total=len(file_list)

print(f'{total}개의 파일에 대해 작업 시작')

for f in tqdm(file_list, total=total) :
    stem, ext = os.path.splitext(f)
    n = stem.strip()
    sn = n[:10]
    
    try :
        e_key = dict_sn[sn]
    except : # 키가 참고 엑셀에 없음
        no_key.append([f])
        continue

    try :
        key = dict_refer[e_key]
    except :
        os.rename(join(path, f), join(noWhere, f))
        noWhere_cnt += 1
        continue

    try :
        new_name = key + " " + n + add + ext
        os.rename(join(path, f), join(path, new_name))
        success.append([f, new_name])
    except Exception as e :
        fail.append([f, e.__class__, e])
        continue

loss = total - len(success) - len(no_key) - noWhere_cnt - len(fail)
print(f'{loss}개 파일 누수')
print(f'{total}개 중 {len(success)}개 파일 이름 변경 완료. {len(no_key)}개 엑셀에 없음, {noWhere_cnt}개 전산없음, {len(fail)}개 파일변경중 에러발생')

write_log_csv(success, path_log, addInfo)
write_log_csv(no_key, path, "noKey")
write_log_csv(fail, path, "fail")

7381개의 파일에 대해 작업 시작


100%|██████████| 7381/7381 [00:05<00:00, 1362.37it/s]

0개 파일 누수
7381개 중 6149개 파일 이름 변경 완료. 12개 엑셀에 없음, 1220개 전산없음, 0개 파일변경중 에러발생
log 파일이 비어있습니다. path : D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류


In [29]:
#####사업자/주민번호 > 채무자키


#  윗셀이랑 혼합되서 원래 코드 지워져버림.



##############################################
path = r'D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류'
addInfo = "광주 모음 대출원장 케이에스 펜타"
noWhere = join(r'D:\전산부재', addInfo)
dict_refer = dict_rgt
add = ""
##############################################
p_rgt1 = re.compile(r'((?<=\D)|\b)\d{13}((?=\D)|\b)') # 13자리
p_rgt2 = re.compile(r'((?<=\D)|\b)(\d{3})-?(\d{2})-?(\d{5})((?=\D)|\b)') # 10자리
no_key = []

p_extension = re.compile('(jpeg|jpg|bmp|gif|pdf|png|tif)$', re.I)
file_list = [f.name for f in os.scandir(path) if f.is_file() and (re.search(p_extension, f.name))]
total=len(file_list)
cnt = 0

print(f'{total}개의 파일에 대해 작업 시작')

for f in tqdm(file_list, total=total) :
    stem, ext = os.path.splitext(f)

    try :
        key = dict_refer[e_key]
    except :
        no_key.append([f, "noDictKey"]) # 전산에 없음
        pass
    try :
        new_name = key + " " + n + add + ext
        os.rename(join(path, f), join(path, new_name))
        cnt += 1
    except Exception as e :
        print(f, e.__class__, e, sep=" : ")
        pass
loss = total - cnt
print(f'{total}개 중 {cnt}개 파일 이름 변경 완료. {loss}개 확인 요망')
write_log_csv(no_key, path)

7381개의 파일에 대해 작업 시작


  1%|          | 85/7381 [00:00<00:08, 846.26it/s]

6001000022 M2_5708111530315_TYM_PENCO_001.pdf : <class 'FileExistsError'> : [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\6001000022 M2_5708111530315_TYM_PENCO_001.pdf' -> 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\20429422 9001001124 M1_6503251465213_BSC_PENCO_002.pdf'
6001000022 N2_3901262530326_TYM_PENCO_001.pdf : <class 'FileExistsError'> : [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\6001000022 N2_3901262530326_TYM_PENCO_001.pdf' -> 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\20429422 9001001124 M1_6503251465213_BSC_PENCO_002.pdf'
6001000022 N2_6210112530312_TYM_PENCO_001.pdf : <class 'FileExistsError'> : [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\6001000022 N2_6210112530312_TYM_PENCO_001.pdf' -> 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\20429422 9001001124 M1_6503251465213_BSC_PENCO_002.pdf'
6001000084 N2_7609172067438_

 13%|█▎        | 930/7381 [00:00<00:01, 3402.64it/s]

6001000751 06813_2002_444_B_09_000000000000000_12.tif : <class 'FileExistsError'> : [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\6001000751 06813_2002_444_B_09_000000000000000_12.tif' -> 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\20429422 9001001124 M1_6503251465213_BSC_PENCO_002.tif'
6001000751 06813_2002_444_B_09_000000000000000_2.tif : <class 'FileExistsError'> : [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\6001000751 06813_2002_444_B_09_000000000000000_2.tif' -> 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\20429422 9001001124 M1_6503251465213_BSC_PENCO_002.tif'
6001000751 06813_2002_444_B_09_000000000000000_3.tif : <class 'FileExistsError'> : [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\6001000751 06813_2002_444_B_09_000000000000000_3.tif' -> 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\20429422 9001001124 M1_6503251465213_BSC

...

[WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\6001001327 06845_2002_339_4_08_000000000000000_3.tif' -> 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\20429422 9001001124 M1_6503251465213_BSC_PENCO_002.tif'
6001001327 06845_2002_339_4_08_000000000000000_4.tif : <class 'FileExistsError'> : [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\6001001327 06845_2002_339_4_08_000000000000000_4.tif' -> 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\20429422 9001001124 M1_6503251465213_BSC_PENCO_002.tif'
6001001327 06845_2002_339_B_14_000000000000000_1.tif : <class 'FileExistsError'> : [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\6001001327 06845_2002_339_B_14_000000000000000_1.tif' -> 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\20429422 9001001124 M1_6503251465213_BSC_PENCO_002.tif'
6001001327 06845_2002_339_B_99_000000000000000_1.tif : <class 'FileEx

 86%|████████▌ | 6343/7381 [00:01<00:00, 3675.35it/s]

6001001442 02445_2002_748_4_08_000000000000000_1.tif : <class 'FileExistsError'> : [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\6001001442 02445_2002_748_4_08_000000000000000_1.tif' -> 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\20429422 9001001124 M1_6503251465213_BSC_PENCO_002.tif'
6001001442 02445_2002_748_4_08_000000000000000_10.tif : <class 'FileExistsError'> : [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\6001001442 02445_2002_748_4_08_000000000000000_10.tif' -> 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\20429422 9001001124 M1_6503251465213_BSC_PENCO_002.tif'
6001001442 02445_2002_748_4_08_000000000000000_11.tif : <class 'FileExistsError'> : [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\6001001442 02445_2002_748_4_08_000000000000000_11.tif' -> 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\20429422 9001001124 M1_6503251465213_B

100%|██████████| 7381/7381 [00:02<00:00, 3622.20it/s]

6001001702 08590_2002_9414_D_01_000000000000000_7.tif : <class 'FileExistsError'> : [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\6001001702 08590_2002_9414_D_01_000000000000000_7.tif' -> 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\20429422 9001001124 M1_6503251465213_BSC_PENCO_002.tif'
6001001702 08590_2002_9414_D_01_000000000000000_8.tif : <class 'FileExistsError'> : [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\6001001702 08590_2002_9414_D_01_000000000000000_8.tif' -> 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\20429422 9001001124 M1_6503251465213_BSC_PENCO_002.tif'
6001001705 02445_2002_10218_4_99_000000000000000_1.tif : <class 'FileExistsError'> : [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\6001001705 02445_2002_10218_4_99_000000000000000_1.tif' -> 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\20429422 9001001124 M1_65032514652

In [7]:
#### 관리자번호
##############################################
path = r'D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류\새 폴더'
addInfo = "광주 모음 대출원장 케이에스 펜타"
noWhere = join(r'D:\전산부재', addInfo)
dict_refer = dict_admin
add = ""
##############################################
p_admin = re.compile(r'((?<=\D)|\b)\d{10}((?=\D)|\b)') # 10자리
no_key = []
log = []
noWhere_cnt = 0
p_extension = re.compile('(jpeg|jpg|bmp|gif|pdf|png|tif)$', re.I)
file_list = [f.name for f in os.scandir(path) if f.is_file() and (re.search(p_extension, f.name))]
total=len(file_list)
cnt = 0

if not os.path.exists(noWhere) :
    os.makedirs(noWhere)

for f in tqdm(file_list, total=total) :
    stem, ext = os.path.splitext(f)
    n = stem.strip()
    if p_admin.search(n) :
        e_key = p_admin.search(n).group()
        # n = p_admin.sub("", n)
    else :
        no_key.append([f, "noSearch"]) # 파일명에 키 없음
        continue

    try :
        key = dict_refer[e_key]
    except :
        os.rename(join(path, f), join(noWhere, f))
        noWhere_cnt += 1
        continue

    try :
        new_name = key + " " + n + add + ext
        log.append(re_name(join(path, f), join(path, new_name)))
        cnt += 1
    except Exception as e :
        print(f, e.__class__, e, sep=" : ")
        continue
loss = total - cnt - noWhere_cnt - len(no_key)
print(f'{loss}개 파일 누수')
print(f'{total}개 중 {cnt}개 파일 이름 변경 완료. {noWhere_cnt}개 전산 없음. {len(no_key)}개 파일명에 키 없음')
write_log_csv(log, path_log, addInfo)

100%|██████████| 2853/2853 [00:00<00:00, 3492.79it/s]

2853개 중 0개 파일 이름 변경 완료. 2853개 확인 요망
log 파일이 비어있습니다. path : D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류\새 폴더
log 파일이 비어있습니다. path : D:\0.전산\1.진행중과업\파일서버개편\log


In [80]:
dict_account['TYM-13-1-00709-01']

KeyError: 'TYM-13-1-00709-01'

In [55]:
# with open(r"D:\0.전산\1.진행중과업\파일서버개편\파일\extraKey\dict모음\TYM.pkl", "wb") as f:  #빈파일 만들고
#     pickle.dump(dict_TYM, f) #객체a를 f에 모두 버려(넘겨)

##### 수정일을 파일명에

In [8]:
def getMtime(path):
    """
    (pdf)수정일=최초생성일!! 이거 써(이동/파일명 변경해도 안 바뀜)
    path : (walk)dir + file 또는 (listdir) file
    """
    a = time.ctime(os.path.getmtime(path))
    b = time.strptime(a)
    c = time.strftime('%Y-%m-%d', b)
    return c

In [12]:
path = r'D:\구 스캔파일_key\extraKey\광주 공유폴더\■양도통지서 스캔\양통모음\김대홍 양통\10.13김대홍양통'
file_list = [f.name for f in os.scandir(path) if f.is_file() and (re.search(p_extension, f.name))]
total=len(file_list)
cnt = 0
for f in tqdm(file_list, total=total) :
    stem = os.path.splitext(f)[0]
    ext = os.path.splitext(f)[1]
    mtime = getMtime(join(path,f))
    new_name = stem + " " + mtime + ext
    try :
        os.rename(join(path, f), join(path, new_name))
        cnt += 1
    except Exception as e :
        print(f, e.__class__, e, sep=" : ")
        pass



100%|██████████| 109/109 [00:00<00:00, 3636.49it/s]


### 회사 코드
##### 펜타

In [58]:
path = r"D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\양통"
file_list = file_listFnc(path)
files = pd.Series(file_list)
files[0]

'20409094 8101001694 M1_6305202796617_WRB_Wfni_001.pdf'

In [59]:
files1 = files.str.replace("M[0-9]", "", regex=True)
files1[0]

'20409094 8101001694 _6305202796617_WRB_Wfni_001.pdf'

In [60]:
files1 = files1.str.replace("N[0-9]", "보증인", regex=True)

In [61]:
files1 = files1.str.replace("KAC-[가나다라마바사아자차카타파하]", "KAC", regex=True)

In [71]:
df = pd.concat([files, files1], axis=1)
df.columns = ["origin_name", "new_name"]
df.head()

,origin_name,new_name
0,20409094 8101001694 M1_6305202796617_WRB_Wfni_...,20409094 8101001694 _6305202796617_WRB_Wfni_00...
1,20409094 8101001694 M3_6305202796617_DNP_PENCO...,20409094 8101001694 _6305202796617_DNP_PENCO_0...
2,20409094 8101001694 M3_6305202796617_DNP_PENCO...,20409094 8101001694 _6305202796617_DNP_PENCO_0...
3,20421357 8101001323 M1_6807141657411_WRB_Wfni_...,20421357 8101001323 _6807141657411_WRB_Wfni_00...
4,20421357 8101001323 M2_6807141657411_Wfni_DNP_...,20421357 8101001323 _6807141657411_Wfni_DNP_00...


In [73]:
df.origin_name[0], df.new_name[0]

('20409094 8101001694 M1_6305202796617_WRB_Wfni_001.pdf',
 '20409094 8101001694 _6305202796617_WRB_Wfni_001.pdf')

In [74]:
addInfo = "광주 모음 대출원장 케이에스 펜타"
success, fail = [], []
total = len(df.index)
for k, v in tqdm(df.iterrows(), total=total) :
    src = join(path, str(v.origin_name))
    dst = join(path, str(v.new_name))

    if src != dst :
        try : 
            os.rename(src, dst)
            success.append([src, dst])
        except Exception as e :
            fail.append([src, dst, e.__class__])


print(f"{total}개 중 {len(success)}개 파일이름 변경, {len(fail)}개 파일 이름 변경 실패")
write_log_csv(success, path_log, addInfo+"양통이름변환")
write_log_csv(fail, path, "실패")